In [18]:
import pandas as pd
import numpy as np

vif = pd.read_csv("VIF.csv", low_memory=False)
vif.columns = vif.columns.str.strip().str.upper()
vif['DEPARTAMENTO'] = vif['HEC_DEPTOMCPIO'].astype(str).str[:2].astype(int)
vif['ANIO'] = vif['HEC_ANO']

vif_agg = vif.groupby(['DEPARTAMENTO','ANIO']).agg(
    edad_victima=('VIC_EDAD', lambda s: s.replace(999, np.nan).mean()),
    edad_agresor=('AGR_EDAD', lambda s: s.replace(999, np.nan).mean()),
    pct_vic_sin_esc=('VIC_ESCOLARIDAD', lambda s: (s==9).mean()),
    pct_agr_sin_esc=('AGR_ESCOLARIDAD', lambda s: (s==9).mean()),
    pct_vic_sin_ocup=('VIC_OCUP', lambda s: (s==99).mean()),
    pct_agr_sin_ocup=('AGR_OCUP', lambda s: (s==99).mean()),
    pct_no_denuncia=('INST_DENUN_HECHO', lambda s: (s!=1).mean()),
    pct_sin_medida=('TIPO_MEDIDA', lambda s: (s==0).mean()),
    total_vif=('HEC_ANO','count')
).reset_index()

div = pd.read_csv("Divorcios.csv", low_memory=False)
div.columns = div.columns.str.strip().str.upper()
div['DEPARTAMENTO'] = div['DEPOCU']
div['ANIO'] = div['AÑOREG']

div_agg = div.groupby(['DEPARTAMENTO','ANIO']).agg(
    edad_hombre=('EDADHOM', lambda s: s.replace(999, np.nan).mean()),
    edad_mujer=('EDADMUJ', lambda s: s.replace(999, np.nan).mean()),
    pct_hom_sin_esc=('ESCHOM', lambda s: (s==9).mean()),
    pct_muj_sin_esc=('ESCMUJ', lambda s: (s==9).mean()),
    total_divorcios=('DEPOCU','count')
).reset_index()

df = pd.merge(div_agg, vif_agg, on=['DEPARTAMENTO','ANIO'], how='outer')
df.columns = df.columns.str.strip().str.upper()


df['TOTAL_DIVORCIOS'] = df['TOTAL_DIVORCIOS'].fillna(0).astype(int)
df['TOTAL_VIF']        = df['TOTAL_VIF'].fillna(0).astype(int)

proportion_cols = [col for col in df.columns if col.startswith('PCT_')]
df[proportion_cols] = df[proportion_cols].fillna(0)

df = df[(df['TOTAL_DIVORCIOS'] > 0) | (df['TOTAL_VIF'] > 0)].reset_index(drop=True)
df = df[(df['ANIO'] >= 2009) & (df['ANIO'] <= 2018)]

df.to_csv("dataset_unificado_limpio.csv", index=False, encoding='utf-8-sig')
print("✅ Dataset final limpio guardado como dataset_unificado_limpio.csv")

✅ Dataset final limpio guardado como dataset_unificado_limpio.csv
